## Function Calling / Tool Calling with vLLM

In this notebook, we will explore how the LLM can use external tools to enhance its capabilities. We'll begin by adding and configuring a search tool to allow the LLM to perform real-time searches.

By the end of this notebook, you'll be able to integrate and configure external search tools with a language model, and use them to perform real-time searches and enhance responses.

### Model Used: Granite-3.1(8B) with Function Calling enabled

## 1. Setup

#### Installing Required Packages

In [1]:
!pip install -q langchain openai "langchain-core==0.3.27" termcolor langchain_community duckduckgo_search wikipedia langchain_experimental langchain_openai


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Imports
import os
import json
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_community.llms import VLLMOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain_core.tools import tool
from typing import Annotated

## 2. DuckDuckGo Search Tool

First, we will configure the tool by initializing the DuckDuckGo search functionality. This will be the primary tool for real-time search queries in this example.

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGo Search Tool
duckduckgo_search = DuckDuckGoSearchRun()

# Adding the search tool to the list of available tools
tools = [duckduckgo_search]

Render the tool’s description and ensures it is formatted correctly for integration into the LLM's prompt.


In [4]:
# Render tool description to ensure it's ready for LLM usage
from langchain.tools.render import render_text_description_and_args

tools_name = duckduckgo_search.name

tools_description = (
    render_text_description_and_args(tools).replace("{", "{{").replace("}", "}}")
)
print("Tools Name:\n", tools_name)
print("Tools Description:\n", tools_description)

Tools Name:
 duckduckgo_search
Tools Description:
 duckduckgo_search - A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query., args: {{'query': {{'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}}}


## 3. Model Configuration

#### Define the Inference Model Server specifics

In [5]:
INFERENCE_SERVER_URL = os.getenv('API_URL_GRANITE')
MODEL_NAME = "granite-3-8b-instruct"
API_KEY= os.getenv('API_KEY_GRANITE')

#### Create the LLM instance

In [6]:
# LLM definition
llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# 4. Tool Calling

Tool calling enables a chat model to generate structured output by "calling a tool" in response to a prompt.

It’s important to note that the model itself doesn’t execute the tool, it only generates the arguments needed for the tool. Running the tool, or deciding not to, is entirely up to the user.

This technique is versatile and can be used even when no tools are actually invoked. LangChain provides standard interfaces for defining tools, integrating them with LLMs, and managing tool calls seamlessly.

In [7]:
llm_with_tools = llm.bind_tools(tools, tool_choice="auto")

For a model to be able to call tools, we need to pass in tool schemas that describe what the tool does and what it's arguments are. Chat models that support tool calling features implement a .bind_tools() method for passing tool schemas to the model.
The tool_choice in the bind_tools() defines which tool to require the model to call, and "auto" automatically selects a tool (including no tool if it's not needed).

## Passing tool outputs to chat models

Now, let's get the model to call a tool. We'll add it to a list of messages that we'll treat as conversation history:

In [8]:
from langchain_core.messages import HumanMessage

query = "Search what is the latest version of OpenShift?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'duckduckgo_search', 'args': {'query': 'latest version of OpenShift'}, 'id': 'chatcmpl-tool-bba88a9c934c420e8509a4c5e8f959f6', 'type': 'tool_call'}]


As you can see the type of message is a "tool_call", and includes the name of the tool used (duckduckgo_search) and the query included in the message as well.

Next, let's use the arguments generated by the model to invoke the tool functions!

With LangChain, invoking a tool using a ToolCall automatically returns a ToolMessage, which can then be seamlessly passed back to the model:

In [9]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"duckduckgo_search": duckduckgo_search}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

/opt/app-root/lib64/python3.9/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


[HumanMessage(content='Search what is the latest version of OpenShift?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'chatcmpl-tool-bba88a9c934c420e8509a4c5e8f959f6', 'function': {'arguments': '{"query": "latest version of OpenShift"}', 'name': 'duckduckgo_search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'granite30-8b'}, id='run-641a2bf1-ed1a-452b-8c64-c1d75d999e0c-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'latest version of OpenShift'}, 'id': 'chatcmpl-tool-bba88a9c934c420e8509a4c5e8f959f6', 'type': 'tool_call'}]),
 ToolMessage(content="Red Hat OpenShift 4.17 is now generally available. Based on Kubernetes 1.30 and CRI-O 1.30, OpenShift 4.17 features expanded control plane options, increased flexibility for virtualization and networking, new capabilities to leverage generative AI, and continued investment in Red Hat OpenShift Platform Plus.

## Final answer after providing the tool result to the LLM

Finally, we'll provide the tool results to the model. Using this information, the model will generate a final response to the original query.

In [10]:
llm_with_tools.invoke(messages)

The latest version of OpenShift is Red Hat OpenShift 4.17, which was released on [insert date]. This version features expanded control plane options, increased flexibility for virtualization and networking, new capabilities to leverage generative AI, and continued investment in Red Hat OpenShift Platform Plus. It also introduces support for IBM Z® and IBM® LinuxONE release on OpenShift Container Platform 4.17. Red Hat provides three different phases of support: Full Support, Maintenance Support, and Extended Update Support. More information, including how to upgrade to the latest version, is available here.

AIMessage(content='The latest version of OpenShift is Red Hat OpenShift 4.17, which was released on [insert date]. This version features expanded control plane options, increased flexibility for virtualization and networking, new capabilities to leverage generative AI, and continued investment in Red Hat OpenShift Platform Plus. It also introduces support for IBM Z® and IBM® LinuxONE release on OpenShift Container Platform 4.17. Red Hat provides three different phases of support: Full Support, Maintenance Support, and Extended Update Support. More information, including how to upgrade to the latest version, is available here.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'granite30-8b'}, id='run-f67d738f-bacd-4abe-9a4c-0cbf31474537-0')

## Conclusion
 
In this notebook, we've successfully extended the capabilities of the LLM by integrating an external tool (DuckDuckGo search). We covered:

- **LLM and Tool Integration:** How to modify the system prompt for tool access.
- **Tool Setup and Configuration:** Setting up the DuckDuckGo search tool for real-time information retrieval.
- **Making Requests and Handling Responses:** Using the tool during LLM interactions and processing the results.

With these tools in place, you're now equipped to extend the LLM's capabilities even further by adding more tools and refining its behavior. Happy coding!